In [ ]:
import keras
import numpy as np

path = keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower() #소문자로
print('length  of corpus : ', len(text))

606208/600901 [==============================] - 0s 0us/step
length  of corpus :  600893


In [ ]:
#글자 시퀀스 벡터화 하기
maxlen = 60
step = 3

sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i:i+maxlen])
  next_chars.append(text[i+maxlen])

print('Number of sequences : ', len(sentences))

chars = sorted(list(set(text)))
print('고유한 글자: ', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorize...')

x = np.zeros((len(sentences),maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
  y[i, char_indices[next_chars[i]]] = 1

Number of sequences :  200278
고유한 글자:  57
Vectorize...


In [ ]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [ ]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)#원핫 인코딩 되어있으니까 범주형 크로스 엔트로피

In [ ]:
# 코드 8-6 모델의 예측이 주어졌을 떄 새로운 글자를 샘플링하는 함수
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [ ]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

# 60 에포크 동안 모델을 훈련합니다
for epoch in range(1, 60):
    print('에포크', epoch)
    # 데이터에서 한 번만 반복해서 모델을 학습합니다
    model.fit(x, y, batch_size=128, epochs=1)

    # 무작위로 시드 텍스트를 선택합니다
    seed_text = text[start_index: start_index + maxlen]
    print('--- 시드 텍스트: "' + seed_text + '"')

    # 여러가지 샘플링 온도를 시도합니다
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)

        # 시드 텍스트에서 시작해서 400개의 글자를 생성합니다
        for i in range(400):
            # 지금까지 생성된 글자를 원-핫 인코딩으로 바꿉니다
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            # 다음 글자를 샘플링합니다
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에포크 1
1565/1565 [==============================] - 288s 183ms/step - loss: 2.2430
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the speak to the sade of the same in the species the same of the self-can the self-can the spirits of the same the same the presention of the same the greeks and any the self-conce are such the nature, and the sangerous of the self-can to the same the self-cas are the same of the self-cate of the self-cas and the same such the manker the same of the same the same in the same the same in the self
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for the mad betone we the speak and the contractity which his cas a spicion and the sate a matter a such the same have man
in the preat, the preach the strenger in the evering something in the can that he langeration of the sadcis of great at the soul is even bene
such that which is to have se

KeyboardInterrupt: ignored